In [1]:
import numpy as np
from PIL import Image
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

########
##Image Processing
########

cpath='TajMahal.jpg'
spath='VGV.jpg'
final='TajVGV.jpg'

height = 512
width  = 512
dim=(height,width)

cimg_original=Image.open('TajMahal.jpg')
cimg=load_img(path=cpath,target_size=dim)
carr= img_to_array(cimg)
carr=K.variable(preprocess_input(np.expand_dims(carr,axis=0)), dtype='float32')

simg_original=Image.open('VGV.jpg')

simg=load_img(path=spath,target_size=dim)
sarr=img_to_array(simg)
sarr=K.variable(preprocess_input(np.expand_dims(sarr,axis=0)), dtype='float32')

gimg = np.random.randint(256,size=(width,height,3)).astype('float64')
garr=img_to_array(gimg)
gimg = (preprocess_input(np.expand_dims(gimg,axis=0)))


gpholder = K.placeholder(shape=(1,height,width,3))

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
########
## Utility Functions
########

def get_f_rep(x,layer_names,model):
    f_matrices=[]
    for layer in layer_names:
        selected_layer = model.get_layer(layer)
        feat_raw = selected_layer.output
        feat_raw_shape = K.shape(feat_raw).eval(session=tf_session)
        Nl = feat_raw_shape[-1]
        Ml = feat_raw_shape[1]*feat_raw_shape[2]
        f_matrix = K.reshape(feat_raw,(Ml,Nl))
        f_matrix = K.transpose(f_matrix)
        f_matrices.append(f_matrix)
    return f_matrices

def content_loss(F,P):
    return 0.5*(K.sum(K.square(F-P)))

def Gram_Matrix(F):
    return K.dot(F,K.transpose(F))

def style_loss(ws,Gs,As):
    sL=K.variable(0)
    
    for (w,G,A) in zip(ws,Gs,As):
        Ml=K.int_shape(G)[1]
        Nl=K.int_shape(G)[0]
        
        G_gram=Gram_Matrix(G)
        A_gram=Gram_Matrix(A)
        
        sL+=w*(0.25)*(K.sum(K.square(G_gram-A_gram)))/(Nl**2 * Ml**2)
    return sL

def total_loss(gpholder,alpha=1.0,beta=10000.0):
    
    F=get_f_rep(gpholder,layer_names=[cLayerName],model=gModel)[0]
    Gs=get_f_rep(gpholder,layer_names=sLayerNames,model=gModel)
    
    s_loss=style_loss(ws,Gs,As)
    c_loss=content_loss(F,P)
    total_loss=alpha*s_loss+beta*c_loss
    return total_loss

def calculate_loss(garr):
    """
     total loss 
    """
    if garr.shape != (1, width, height, 3):
        garr = garr.reshape((1, width, height, 3))
    loss_fcn = K.function([gModel.input], [total_loss(gModel.input)])
    loss = (loss_fcn([garr])[0].astype('float64'))
    
    return loss

def get_grad(garr):
    """
     gradient of the loss function with respect to the generated image
    """
    if garr.shape != (1, width, height, 3):
        garr = garr.reshape((1, width, height, 3))
    grad_fcn = K.function([gModel.input], K.gradients(total_loss(gModel.input), [gModel.input]))
    grad = grad_fcn([garr])[0].flatten().astype('float64')
    return grad

def postprocess_array(x):
    # Zero-center by mean pixel
    if x.shape != (width, height, 3):
        x = x.reshape((width, height, 3))
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

def reprocess_array(x):
    x = np.expand_dims(x.astype('float64'), axis=0)
    x = preprocess_input(x)
    return x

def save_original_size(x, target_size=dim):
    xIm = Image.fromarray(x)
    xIm = xIm.resize(target_size)
    xIm.save('TAJVGV.jpeg','jpeg')
    return xIm

In [4]:
tf_session = K.get_session()
cModel = VGG16(include_top=False, weights='imagenet', input_tensor=carr)
sModel = VGG16(include_top=False, weights='imagenet', input_tensor=sarr)
gModel = VGG16(include_top=False, weights='imagenet', input_tensor=gpholder)
cLayerName = 'block4_conv2'
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                #'block5_conv1'
                ]

P = get_f_rep(x=carr, layer_names=[cLayerName], model=cModel)[0]
As = get_f_rep(x=sarr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))

iterations = 10

In [ ]:
x_val = gimg.flatten()

start = time.time()
"""xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
                            maxiter=iterations, disp=True)"""


result_prefix = 'style_transfer_result'
for i in range(iterations):
    print('Start of iteration', i)
    starttime = time.time()
    x, f_val, info = fmin_l_bfgs_b(calculate_loss, x_val,
                                     fprime=get_grad, maxfun=20)
    
    print('Current loss value:', f_val)
    # Save current generated image
    img = x.copy().reshape((height, width, 3))
    img = postprocess_array(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    endtime = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, endtime - starttime))

In [ ]:
from matplotlib import pyplot as plt

# Content image
plt.imshow(cimg)
plt.figure()

# Style image
plt.imshow(simg)
plt.figure()

# Generate image
plt.imshow(img)
plt.show()